# Regression : LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

Make sure you have the latest version of scikit-learn

In [122]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [123]:
sales = pd.read_csv('kc_house_data.csv')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
#sales['floors'] = sales['floors'].astype(int) 
sales

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
5,7237550310,20140512T000000,1225000.0,4,4.50,5420,101930,1.0,0,0,...,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
6,1321400060,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
7,2008000270,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711
8,2414600126,20150415T000000,229500.0,3,1.00,1780,7470,1.0,0,0,...,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113
9,3793500160,20150312T000000,323000.0,3,2.50,1890,6560,2.0,0,0,...,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570


If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

In [124]:
sales.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

# Import useful functions from previous notebook

As in Week 2, we convert the DataFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [125]:
x = sales.drop(['id','date','price'] , axis =1)
y = sales['price']
print(x.shape ,y.shape)

(21613, 18) (21613,)


In [126]:
def get_numpy_data(data_frame,features,output):
    data_frame['constant'] = 1
    features = ['constant'] + features
    feature_matrix = data_frame[features]
    output_matrix = output
    
    return (feature_matrix,output_matrix)


Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [127]:
def predict_output(feature_matrix,weights):
    #predict=np.dot(weights,feature_matrix)
    predict=np.dot(feature_matrix,weights)

    return predict

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [128]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print (X.shape ,X)

(2, 3) [[ 3.  5.  8.]
 [ 4. 12. 15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [129]:
norms_0 = np.linalg.norm(X, axis=0) 
# axis = 0 is for column wise operation 
norms_1 = np.linalg.norm(X, axis=1) 
# axis = 1 is for row wise operation
# gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print (norms_0,norms_1)

[ 5. 13. 17.] [ 9.89949494 19.62141687]


To normalize, apply element-wise division:

In [130]:
print (X / norms_0)
# gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

#print (X / norms_1) 
# operands could not be broadcast together with shapes (2,3) (2,) 


[[0.6        0.38461538 0.47058824]
 [0.8        0.92307692 0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [131]:
def normalize_features(feature_matrix):
    
    norms = np.linalg.norm(feature_matrix, axis = 0)
    features = feature_matrix/norms
    
    return (features,norms)

To test the function, run the following:

In [132]:
a = np.array([[3.,6.,9.],[4.,8.,12.]])
features, norms = normalize_features(a)
print (features)
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print (norms)
# should print
# [5.  10.  15.]

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]]
[ 5. 10. 15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [133]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(x, simple_features, y)

Don't forget to normalize features:

In [134]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)
print(simple_feature_matrix  ,norms)

       constant  sqft_living  bedrooms
0      0.006802     0.003530  0.005836
1      0.006802     0.007689  0.005836
2      0.006802     0.002304  0.003890
3      0.006802     0.005864  0.007781
4      0.006802     0.005026  0.005836
5      0.006802     0.016215  0.007781
6      0.006802     0.005131  0.005836
7      0.006802     0.003171  0.005836
8      0.006802     0.005325  0.005836
9      0.006802     0.005654  0.005836
10     0.006802     0.010650  0.005836
11     0.006802     0.003470  0.003890
12     0.006802     0.004278  0.005836
13     0.006802     0.004099  0.005836
14     0.006802     0.005415  0.009726
15     0.006802     0.008826  0.007781
16     0.006802     0.005654  0.005836
17     0.006802     0.004787  0.007781
18     0.006802     0.003590  0.003890
19     0.006802     0.003740  0.005836
20     0.006802     0.004847  0.007781
21     0.006802     0.009125  0.005836
22     0.006802     0.006791  0.009726
23     0.006802     0.003201  0.003890
24     0.006802     0.007

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [135]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [136]:
prediction = predict_output(simple_feature_matrix,weights)
print(prediction)

[0.02675867 0.04339256 0.01990703 ... 0.02289873 0.03178473 0.02289873]


Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [137]:
def get_ro(simple_feature_matrix, output, weights, i):
    
    prediction = predict_output(simple_feature_matrix, weights)
    feature_i = simple_feature_matrix.T
    #print(feature_i)
    ro_i = feature_i * (output - prediction + weights[i]*feature_i) 
    #print(ro_i)
    ro_i = np.sum(ro_i).T
    
    return ro_i[i] 

Below are step wise explanation of ro_i 

In [138]:
pred_i = predict_output(simple_feature_matrix[:20], weights)
print(pred_i.shape)
feat_i = simple_feature_matrix[:20].T
print(feat_i)

i = 0
feat_wgt = feat_i*weights[i]
print(feat_wgt.shape)
out_i = output[:20]
in_diff = (out_i - feat_wgt + pred_i)
print(in_diff.shape ,in_diff)
ro_ = (feat_i * in_diff)
#print(ro_i)
ro_ = np.sum (ro_ ).T
print(ro_)
    

(20,)
                   0         1         2         3         4         5   \
constant     0.006802  0.006802  0.006802  0.006802  0.006802  0.006802   
sqft_living  0.003530  0.007689  0.002304  0.005864  0.005026  0.016215   
bedrooms     0.005836  0.005836  0.003890  0.007781  0.005836  0.007781   

                   6         7         8         9         10        11  \
constant     0.006802  0.006802  0.006802  0.006802  0.006802  0.006802   
sqft_living  0.005131  0.003171  0.005325  0.005654  0.010650  0.003470   
bedrooms     0.005836  0.005836  0.005836  0.005836  0.005836  0.003890   

                   12        13        14        15        16        17  \
constant     0.006802  0.006802  0.006802  0.006802  0.006802  0.006802   
sqft_living  0.004278  0.004099  0.005415  0.008826  0.005654  0.004787   
bedrooms     0.005836  0.005836  0.009726  0.007781  0.005836  0.007781   

                   18        19  
constant     0.006802  0.006802  
sqft_living  0.003590  

In [139]:
out_i[:20]

0      221900.0
1      538000.0
2      180000.0
3      604000.0
4      510000.0
5     1225000.0
6      257500.0
7      291850.0
8      229500.0
9      323000.0
10     662500.0
11     468000.0
12     310000.0
13     400000.0
14     530000.0
15     650000.0
16     395000.0
17     485000.0
18     189000.0
19     230000.0
Name: price, dtype: float64

In [140]:
# its very delicate with handling with shapes , and operations 

***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

In [141]:
print ('ro_1', get_ro(simple_feature_matrix, output, weights, 1) )
print ('ro_2', get_ro(simple_feature_matrix, output, weights, 2) )

ro_1 10935.653766654435
ro_2 2339.312700739596


In [142]:
print ('ro_1', get_ro(simple_feature_matrix, output, weights, 0) )
print ('ro_2', get_ro(simple_feature_matrix, output, weights, 1) )

ro_1 3587.683524067428
ro_2 10935.653766654435


***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

In [143]:
r_i = get_ro(simple_feature_matrix, output, weights, 1)
r_i

10935.653766654435

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [144]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    #predictions is not need because it is there in get_ro 
    #prediction = predict_output(feature_matrix,weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = get_ro(feature_matrix, output, weights,i)

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [145]:
# should print 0.425558846691
import math
#x_1 =np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)] ] )
x_1 = pd.DataFrame([[1,2,3],[2,3,2]]).T
wg = np.array([1,2])
po = predict_output(x_1 ,wg)
op = [22,11,15] 
print(x_1,po,op)      

ro_i = get_ro(x_1, op , wg ,0)
ro_i

   0  1
0  1  2
1  2  3
2  3  2 [5 8 7] [22, 11, 15]


56

## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [146]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = initial_weights
    max_weights_change = tolerance
    while (max_weights_change >= tolerance):
        max_weights_change = 0
        for i in range(len(weights)):
            old_weights_i = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            weights_change = abs(old_weights_i - weights[i])
            if weights_change > max_weights_change:
                max_weights_change = weights_change
    return weights

    

Using the following parameters, learn the weights on the sales dataset. 

In [147]:
simple_features = ['bedrooms','bathrooms','sqft_living', 'sqft_lot','floors','waterfront', 
                   'view', 'condition', 'grade','sqft_above','sqft_basement','yr_built', 'yr_renovated']

In [148]:
#simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(14)
l1_penalty = 1e-7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [149]:
(simple_feature_matrix, output) = get_numpy_data(x, simple_features, y)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [150]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
weights

array([ 10390.74454471,  54155.48268459,   7554.56529261,  43705.76280672,
        27846.55890773, 146647.43093546,  15048.3702661 ,  14096.87810952,
        13243.60942378,  19511.49116626,  55838.17365972,  23273.38696248,
        16932.76346703,  21240.82506629])

In [151]:
print (weights)
#rss on normalized data
prediction = predict_output(normalized_simple_feature_matrix, weights)
print(prediction)
residuals = prediction - output
#print('residuals::',    residuals)
rss_normalized_data = (residuals * residuals).sum()
print (rss_normalized_data)

[ 10390.74454471  54155.48268459   7554.56529261  43705.76280672
  27846.55890773 146647.43093546  15048.3702661   14096.87810952
  13243.60942378  19511.49116626  55838.17365972  23273.38696248
  16932.76346703  21240.82506629]
[1755.3501121  3611.20137454 1517.64979898 ... 2205.02907202 2559.24740501
 2203.79347328]
9149866119586256.0


***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [152]:
from sklearn.model_selection import train_test_split
x_train,x_test ,y_train ,y_test = train_test_split(x , y , test_size=.2,random_state=20)
print(x_train.shape ,y_train.shape)
print(x_test.shape ,y_test.shape)

(17290, 19) (17290,)
(4323, 19) (4323,)


Let us consider the following set of features.

In [153]:
all_features = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront', 
                'view','condition', 'grade','sqft_above','sqft_basement','yr_built','yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [154]:
(all_feature_matrix, train_output) = get_numpy_data(x_train, all_features, y_train)
(normalized_all_feature_matrix, train_norms) = normalize_features(all_feature_matrix)

C:\Users\Srikanth\AppData\Local\conda\conda\envs\SRI\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [155]:
normalized_all_feature_matrix

,constant,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated
8184,0.007605,0.004348,0.005061,0.003909,0.000180,0.014342,0.0,0.000000,0.00657,0.006869,0.004513,0.000000,0.007727,0.000000
3291,0.007605,0.004348,0.003374,0.002739,0.000814,0.004781,0.0,0.000000,0.00876,0.005888,0.003163,0.000000,0.007426,0.000000
9192,0.007605,0.004348,0.008435,0.004176,0.000264,0.009561,0.0,0.000000,0.00657,0.006869,0.003973,0.003147,0.007735,0.000000
17517,0.007605,0.006522,0.005904,0.004143,0.001437,0.004781,0.0,0.000000,0.00876,0.006869,0.004783,0.000000,0.007611,0.000000
7514,0.007605,0.006522,0.005904,0.008051,0.001492,0.004781,0.0,0.000000,0.00876,0.007850,0.007175,0.007867,0.007588,0.000000
9337,0.007605,0.006522,0.007591,0.006114,0.000791,0.009561,0.0,0.000000,0.00657,0.006869,0.007059,0.000000,0.007704,0.000000
910,0.007605,0.006522,0.005904,0.004744,0.001458,0.004781,0.0,0.000000,0.00876,0.006869,0.005477,0.000000,0.007584,0.000000
4743,0.007605,0.010871,0.011808,0.012094,0.001305,0.009561,0.0,0.018946,0.00657,0.009813,0.010762,0.011872,0.007554,0.037195
18645,0.007605,0.002174,0.003374,0.004376,0.001446,0.007171,0.0,0.000000,0.00219,0.005888,0.005053,0.000000,0.007399,0.000000
9845,0.007605,0.006522,0.007591,0.007884,0.002494,0.004781,0.0,0.000000,0.00876,0.008832,0.009103,0.000000,0.007631,0.000000


In [156]:
train_output.shape

(17290,)

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [157]:
l = len(all_features)
in_weights = np.zeros(l+1)
l1_penalty = 1e-7
tolerance = 1.0
print(in_weights)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [158]:
weights1e7 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, train_output, in_weights, l1_penalty, tolerance)
weights1e7_normalized = weights1e7 / train_norms
print (weights1e7_normalized)

KeyError: 1

***QUIZ QUESTION***

What features had non-zero weight in this case?

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [ ]:
weights1e8 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, train_output, in_weights, 1e8, tolerance)
weights1e8_normalized = weights1e8 / train_norms
print (weights1e8_normalized)

***QUIZ QUESTION***

What features had non-zero weight in this case?

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [ ]:
in_weights

In [ ]:
l1_penalty=1e4
tolerance=5e5
weights1e4 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, train_output, initial_weights, l1_penalty, tolerance)
weights1e4_normalized = weights1e4 / train_norms
print (weights1e4_normalized)

***QUIZ QUESTION***

What features had non-zero weight in this case?

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [ ]:
(test_feature_matrix, test_output) = get_numpy_data(x_test, all_features, y_test)

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [ ]:
def get_rss(features_matrix, weights, output):
    print (weights)
    prediction = predict_output(features_matrix, weights)
    print(prediction)
    residuals = prediction - output
    RSS = (residuals * residuals).sum()
    return RSS

In [ ]:
rss_weights1e4 = get_rss(test_feature_matrix, weights1e4_normalized, test_output)
print (rss_weights1e4)

In [ ]:
rss_weights1e7 = get_rss(test_feature_matrix, weights1e7_normalized, test_output)
print (rss_weights1e7)

In [ ]:

rss_weights1e8 = get_rss(test_feature_matrix, weights1e8_normalized, test_output)
print (rss_weights1e8)

In [ ]:
y_pred = predict_output(test_feature_matrix, weights1e4_normalized)


***QUIZ QUESTION***

Which model performed best on the test data?

In [ ]:
plt.plot(x_test['sqft_living'] , y_test , '.')
plt.plot(x_test['sqft_living'] , y_pred , '*')